In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
import gc
import tensorflow as tf
import holidays

from tensorflow.keras import layers
from tensorflow import keras

from sklearn.model_selection import train_test_split

# Transformer with TIME2VEC

Here I tried a transformer model with almost nothing of preprocessing on the data. The results aren't that good yet, so next time
I am going do some feature engineering, like adding information extracted from the dates, like holidays, weekends and month of the year for instance.

In [ ]:
def smape(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    num = tf.math.abs(tf.math.subtract(y_true, y_pred))
    denom = tf.math.add(tf.math.abs(y_true), tf.math.abs(y_pred))
    denom = tf.math.divide(denom,200.0)
    
    val = tf.math.divide(num,denom)
    val = tf.where(denom == 0.0, 0.0, val) 
    return tf.reduce_mean(val)

In [ ]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
class Time2Vector(tf.keras.layers.Layer):
    def __init__(self, seq_len, **kwargs):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

    def build(self, input_shape):
        self.weights_linear = self.add_weight(name='weight_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_linear = self.add_weight(name='bias_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.weights_periodic = self.add_weight(name='weight_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_periodic = self.add_weight(name='bias_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

    def call(self, x):
        x = tf.math.reduce_mean(x[:,:,:], axis=-1) # Convert (batch, seq_len, 5) to (batch, seq_len)
        time_linear = self.weights_linear * x + self.bias_linear
        time_linear = tf.expand_dims(time_linear, axis=-1) # (batch, seq_len, 1)
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = tf.expand_dims(time_periodic, axis=-1) # (batch, seq_len, 1)
        return tf.concat([time_linear, time_periodic], axis=-1) # (batch, seq_len, 2

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.attn(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class Transformer(keras.Model):
    def __init__(
            self,
            num_hid=64, # embed_dim - num of features
            time_steps=7,
            num_head = 2,
            num_feed_forward=128, # pointwise dim
            num_layers_enc = 4,
            time_embedding = False,
    ):
        super().__init__()
        self.num_hid = num_hid
        if time_embedding:
            self.num_hid += 2
            self.tv = Time2Vector(time_steps)
        else:
            self.tv = None
        self.numlayers_enc = num_layers_enc
        self.enc_input = layers.Input((time_steps, self.num_hid))
        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(self.num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )
        self.GlobalAveragePooling1D = layers.GlobalAveragePooling1D(data_format='channels_last')
        self.out = layers.Dense(units=1, activation='linear')        
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        
    def call(self, inputs):
        if self.tv:
            x = self.tv(inputs)
            x = self.concat([inputs, x])
            x = self.encoder(x)
        else:
            x = self.encoder(inputs)
        x = self.GlobalAveragePooling1D(x)
        y = self.out(x)
        return y

In [ ]:
def label_encoder(df):
    country = {c : i for i, c in enumerate(df['country'].unique())}
    store = {s : i for i, s in enumerate(df['store'].unique())}
    product = {p : i for i, p in enumerate(df['product'].unique())}
    df = df.copy()
    df['country'] = df['country'].replace(country)
    df['store'] = df['store'].replace(store)
    df['product'] = df['product'].replace(product)
    return df

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
def preprocess_dates(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df['date'].dt.weekday
    df['quarter'] = df['date'].dt.quarter
    df['day_of_year'] = df['date'].dt.day_of_year
    df['is_month_start'] = df['date'].dt.is_month_start.astype("int8")
    df['is_month_end'] = df['date'].dt.is_month_end.astype("int8")
    df['month'] = df['date'].dt.month
    return df

In [ ]:
def preprocess_holidays(df):
    holiday_finland = holidays.CountryHoliday(country='FI', years=[2015, 2016, 2017, 2018, 2019])
    holiday_norway = holidays.CountryHoliday(country='NO', years=[2015, 2016, 2017, 2018, 2019])
    holiday_sweden = holidays.CountryHoliday(country='SE', years=[2015, 2016, 2017, 2018, 2019])
    holidays_fin_nor_swe = holiday_finland.copy()
    holidays_fin_nor_swe.update(holiday_norway)
    holidays_fin_nor_swe.update(holiday_sweden)
    dates = list(holidays_fin_nor_swe.keys())
    dates = sorted(pd.to_datetime(dates))
    df = df.copy()
    df['is_holiday'] = df['date'].apply(lambda x : 1 if x in dates else 0)
    return df

In [ ]:
def preprocess_timeseries(df):
    df = df.copy()
    df['sin_day_of_year'] = np.sin(df['day_of_year'])
    df['sin_month'] = np.sin(df['month'])
    return df

In [ ]:
seed = 47
TIMESTEPS = 1
warnings.filterwarnings("ignore")

# Reading the dataset

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/train.csv", sep=',')

# Preprocessing

In [ ]:
train_df = label_encoder(train_df)
train_df = preprocess_dates(train_df)
train_df = preprocess_holidays(train_df)
train_df = preprocess_timeseries(train_df)
x_train = train_df.drop(['row_id', 'date', 'num_sold'], axis=1)
y_train = train_df['num_sold']
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=seed, shuffle=False)

<h3>Converte the data to 3D array shape</h3>

In [ ]:
x_train = np.append(x_train, y_train.values.reshape(-1, 1), axis=1)
x_test = np.append(x_test, y_test.values.reshape(-1, 1), axis=1)
x_train, y_train = split_sequences(x_train, TIMESTEPS)
x_test, y_test = split_sequences(x_test, TIMESTEPS)

In [ ]:
num_heads=2
num_layers_enc=2
num_feed_forward=64
num_features = x_train.shape[-1]
time_steps = TIMESTEPS
epochs = 100
batch_size = 128

model = Transformer(num_hid=num_features,
                        time_steps=time_steps,
                        time_embedding=True,
                        num_head=num_heads,
                        num_layers_enc=num_layers_enc,
                        num_feed_forward=num_feed_forward)

opt = tf.keras.optimizers.Adam()
loss = tf.keras.losses.mse
model.compile(optimizer=opt, loss=smape)
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
print()
results = model.evaluate(x_test, y_test)
print(results)

# Submission

In [ ]:
del train_df, x_train, y_train, x_test, y_test
gc.collect()

In [ ]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/test.csv", sep=',')

In [ ]:
test_df = label_encoder(test_df)
test_df = preprocess_dates(test_df)
test_df = preprocess_holidays(test_df)
test_df = preprocess_timeseries(test_df)
x_test = test_df.drop(['row_id', 'date'], axis=1)
x_test = np.append(x_test, np.ones((x_test.shape[0], 1)), axis=1)
x_test, _ = split_sequences(x_test, TIMESTEPS)

In [ ]:
target = model.predict(x_test).squeeze()
row_id =  test_df['row_id'].values
submission = pd.DataFrame({'row_id' : row_id, 'num_sold' : target})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)